# NBA Master Tables Creation

This notebook creates and maintains the master tables that serve as the foundation for all NBA data pulls:

1. **Master Games**: All NBA games across seasons/leagues
2. **Master Players**: All NBA players across seasons  
3. **Master Teams**: All NBA teams and their details
4. **Master Seasons**: Available seasons and their metadata

These master tables provide the IDs needed for endpoint parameters in the systematic data collection process.

In [1]:
import pandas as pd
import time
import numpy as np
import sys
import os

# NBA API imports
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.static import teams, players

# Database imports
import psycopg2
import re

# Add src directory to path for importing allintwo
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
import allintwo

In [4]:
print("\n=== CREATING MASTER GAMES ===")
# Optimized games collection with error handling
gamesl = []
failed_seasons_games = []

for i, season in enumerate(seasons):
    try:
        print(f"Fetching games for season {season} ({i+1}/{len(seasons)})")
        gamefinder = leaguegamefinder.LeagueGameFinder(
            league_id_nullable='00', 
            season_type_nullable="Regular Season",
            season_nullable=season
        ).get_data_frames()[0]
        
        if len(gamefinder) > 0:
            gamesl.append(gamefinder)
            print(f"  ✓ Retrieved {len(gamefinder)} games")
        else:
            print(f"  ⚠ No games found for {season}")
            
        time.sleep(0.6)  # Rate limiting
        
    except Exception as e:
        print(f"  ✗ Failed to fetch games for {season}: {str(e)}")
        failed_seasons_games.append(season)
        time.sleep(2)  # Longer wait after error

# Combine all games
if gamesl:
    gamehistory = pd.concat(gamesl, axis=0, ignore_index=True)
    gamehistory['GAME_DATE'] = pd.to_datetime(gamehistory['GAME_DATE'])
    print(f"Total games collected: {len(gamehistory)}")
else:
    print("No games data collected!")

print("\n=== CREATING MASTER PLAYERS ===")
# Optimized players collection with error handling  
playersl = []
failed_seasons_players = []

for i, season in enumerate(seasons):
    try:
        print(f"Fetching players for season {season} ({i+1}/{len(seasons)})")
        playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=season).get_data_frames()[0]
        
        if len(playerfinder) > 0:
            playerfinder['season'] = season
            playersl.append(playerfinder)
            print(f"  ✓ Retrieved {len(playerfinder)} players")
        else:
            print(f"  ⚠ No players found for {season}")
            
        time.sleep(0.4)  # Slightly faster for players
        
    except Exception as e:
        print(f"  ✗ Failed to fetch players for {season}: {str(e)}")
        failed_seasons_players.append(season)
        time.sleep(2)  # Longer wait after error

# Combine all players
if playersl:
    allplayers = pd.concat(playersl, axis=0, ignore_index=True)
    print(f"Total player records collected: {len(allplayers)}")
else:
    print("No players data collected!")

# Report any failures
if failed_seasons_games:
    print(f"\nFailed to fetch games for seasons: {failed_seasons_games}")
if failed_seasons_players:
    print(f"Failed to fetch players for seasons: {failed_seasons_players}")

print("\n=== CREATING AND POPULATING DATABASE TABLES ===")

# Drop existing tables to recreate
cursor = conn.cursor()
tables_to_drop = ['mastergames', 'masterplayers', 'masterteams', 'masterseasons']
for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table};")
    print(f"Dropped table: {table}")

conn.commit()

# Clean column names for all datasets
gamehistory = allintwo.clean_column_names(gamehistory)
allplayers = allintwo.clean_column_names(allplayers)
master_teams = allintwo.clean_column_names(master_teams)
master_seasons = allintwo.clean_column_names(master_seasons)

# Create tables
datasets = [
    ('mastergames', gamehistory),
    ('masterplayers', allplayers), 
    ('masterteams', master_teams),
    ('masterseasons', master_seasons)
]

for table_name, df in datasets:
    print(f"Creating table: {table_name} with {len(df)} records")
    allintwo.create_table(conn, table_name, df)
    allintwo.insert_dataframe_to_rds(conn, df, table_name)
    print(f"  ✓ {table_name} completed")

print("\n🎉 All master tables created successfully!")
print("Tables created:", [name for name, _ in datasets])


=== CREATING MASTER GAMES ===
Fetching games for season 2024-25 (1/79)
  ✓ Retrieved 2460 games
  ✓ Retrieved 2460 games
Fetching games for season 2023-24 (2/79)
Fetching games for season 2023-24 (2/79)
  ✓ Retrieved 2460 games
  ✓ Retrieved 2460 games
Fetching games for season 2022-23 (3/79)
Fetching games for season 2022-23 (3/79)
  ✓ Retrieved 2460 games
  ✓ Retrieved 2460 games
Fetching games for season 2021-22 (4/79)
Fetching games for season 2021-22 (4/79)
  ✓ Retrieved 2460 games
  ✓ Retrieved 2460 games
Fetching games for season 2020-21 (5/79)
Fetching games for season 2020-21 (5/79)
  ✓ Retrieved 2160 games
  ✓ Retrieved 2160 games
Fetching games for season 2019-20 (6/79)
Fetching games for season 2019-20 (6/79)
  ✓ Retrieved 2118 games
  ✓ Retrieved 2118 games
Fetching games for season 2018-19 (7/79)
Fetching games for season 2018-19 (7/79)
  ✓ Retrieved 2460 games
  ✓ Retrieved 2460 games
Fetching games for season 2017-18 (8/79)
Fetching games for season 2017-18 (8/79)
  ✓ 

C:\Users\ajwin\AppData\Local\Temp\ipykernel_9204\3771580200.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gamehistory = pd.concat(gamesl, axis=0, ignore_index=True)


Total games collected: 95774

=== CREATING MASTER PLAYERS ===
Fetching players for season 2024-25 (1/79)
  ✓ Retrieved 569 players
  ✓ Retrieved 569 players
Fetching players for season 2023-24 (2/79)
Fetching players for season 2023-24 (2/79)
  ✓ Retrieved 572 players
  ✓ Retrieved 572 players
Fetching players for season 2022-23 (3/79)
Fetching players for season 2022-23 (3/79)
  ✓ Retrieved 539 players
  ✓ Retrieved 539 players
Fetching players for season 2021-22 (4/79)
Fetching players for season 2021-22 (4/79)
  ✓ Retrieved 605 players
  ✓ Retrieved 605 players
Fetching players for season 2020-21 (5/79)
Fetching players for season 2020-21 (5/79)
  ✓ Retrieved 540 players
  ✓ Retrieved 540 players
Fetching players for season 2019-20 (6/79)
Fetching players for season 2019-20 (6/79)
  ✓ Retrieved 529 players
  ✓ Retrieved 529 players
Fetching players for season 2018-19 (7/79)
Fetching players for season 2018-19 (7/79)
  ✓ Retrieved 530 players
  ✓ Retrieved 530 players
Fetching player

AttributeError: 'NoneType' object has no attribute 'cursor'

In [3]:
# Configuration and Setup
print("Starting Master Tables Creation Process...")

# Try database connection
try:
    conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')
    db_connected = conn is not None
    print("✓ Database connected successfully")
except Exception as e:
    print(f"⚠ Database connection failed: {str(e)}")
    print("Continuing with data collection for local validation...")
    db_connected = False
    conn = None

# Generate comprehensive seasons list (1946-47 to current)
def generate_seasons(start_year=1946, end_year=2025):
    """Generate NBA seasons in proper format"""
    seasons = []
    for i in range(start_year, end_year):
        seasons.append(f"{i}-{str(i+1)[2:].zfill(2)}")
    return seasons[::-1]  # Reverse to get most recent first

seasons = generate_seasons()
print(f"Generated {len(seasons)} seasons: {seasons[:5]}...{seasons[-5:]}")

# Create Master Teams table
print("\n=== CREATING MASTER TEAMS ===")
teams_data = teams.get_teams()
master_teams = pd.DataFrame(teams_data)
master_teams_clean = allintwo.clean_column_names(master_teams.copy())
print(f"Retrieved {len(master_teams)} teams")
print(master_teams.head())

# Create Master Seasons table
print("\n=== CREATING MASTER SEASONS ===")
master_seasons = pd.DataFrame({
    'season': seasons,
    'start_year': [int(s.split('-')[0]) for s in seasons],
    'end_year': [int('20' + s.split('-')[1]) if len(s.split('-')[1]) == 2 else int('19' + s.split('-')[1]) for s in seasons],
    'created_date': pd.Timestamp.now()
})
master_seasons_clean = allintwo.clean_column_names(master_seasons.copy())
print(f"Created {len(master_seasons)} seasons")
print(master_seasons.head())

# Save locally for now
print("\n=== SAVING DATA LOCALLY ===")
master_teams.to_csv('../data/master_teams.csv', index=False)
master_seasons.to_csv('../data/master_seasons.csv', index=False)
print("✓ Master teams and seasons saved locally")

Starting Master Tables Creation Process...
Error connecting to RDS: connection to server at "nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com" (44.208.170.145), port 5432 failed: Connection timed out (0x0000274C/10060)
	Is the server running on that host and accepting TCP/IP connections?

✓ Database connected successfully
Generated 79 seasons: ['2024-25', '2023-24', '2022-23', '2021-22', '2020-21']...['1950-51', '1949-50', '1948-49', '1947-48', '1946-47']

=== CREATING MASTER TEAMS ===
Retrieved 30 teams
           id             full_name abbreviation   nickname         city  \
0  1610612737         Atlanta Hawks          ATL      Hawks      Atlanta   
1  1610612738        Boston Celtics          BOS    Celtics       Boston   
2  1610612739   Cleveland Cavaliers          CLE  Cavaliers    Cleveland   
3  1610612740  New Orleans Pelicans          NOP   Pelicans  New Orleans   
4  1610612741         Chicago Bulls          CHI      Bulls      Chicago   

           state  year_f

In [ ]:
print("=== MASTER TABLES VALIDATION ===")

# Test database connection
conn = allintwo.connect_to_rds('thebigone', 'ajwin', 'CharlesBark!23', 'nba-rds-instance.c9wwc0ukkiu5.us-east-1.rds.amazonaws.com')

# Validate all master tables
master_tables = ['mastergames', 'masterplayers', 'masterteams', 'masterseasons']

for table_name in master_tables:
    try:
        df = allintwo.fetch_table_to_dataframe(conn, table_name)
        print(f"\n📊 {table_name.upper()}:")
        print(f"  Records: {len(df):,}")
        print(f"  Columns: {len(df.columns)}")
        
        # Show sample data
        if len(df) > 0:
            print("  Sample data:")
            display(df.head(3))
            
            # Table-specific insights
            if table_name == 'mastergames':
                date_range = pd.to_datetime(df['gamedate'])
                print(f"  Date range: {date_range.min().strftime('%Y-%m-%d')} to {date_range.max().strftime('%Y-%m-%d')}")
                print(f"  Unique games: {df['gameid'].nunique():,}")
                
            elif table_name == 'masterplayers':
                print(f"  Unique players: {df['playerid'].nunique():,}")
                print(f"  Seasons covered: {df['season'].nunique()}")
                
            elif table_name == 'masterteams':
                print(f"  Total teams: {len(df)}")
                
            elif table_name == 'masterseasons':
                print(f"  Season range: {df['season'].min()} to {df['season'].max()}")
        else:
            print("  ⚠ No data found!")
            
    except Exception as e:
        print(f"❌ Failed to validate {table_name}: {str(e)}")

print(f"\n✅ Master tables validation complete!")

Connected to RDS PostgreSQL database
Data fetched successfully from mastergames table.
